In [1]:
!pip freeze

accelerate==1.0.0
aiohttp==3.9.3
aiosignal==1.3.1
asttokens @ file:///opt/conda/conda-bld/asttokens_1646925590279/work
astunparse==1.6.3
async-timeout==4.0.3
attrs==23.1.0
backcall @ file:///home/ktietz/src/ci/backcall_1611930011877/work
beautifulsoup4 @ file:///croot/beautifulsoup4-split_1681493039619/work
boltons @ file:///croot/boltons_1677628692245/work
brotlipy==0.7.0
certifi @ file:///home/conda/feedstock_root/build_artifacts/certifi_1725278078093/work/certifi
cffi @ file:///croot/cffi_1670423208954/work
chardet @ file:///home/builder/ci_310/chardet_1640804867535/work
charset-normalizer @ file:///tmp/build/80754af9/charset-normalizer_1630003229654/work
click @ file:///tmp/build/80754af9/click_1646056706450/work
comm @ file:///home/conda/feedstock_root/build_artifacts/comm_1710320294760/work
conda @ file:///croot/conda_1696257509808/work
conda-build @ file:///croot/conda-build_1696257509796/work
conda-content-trust @ file:///croot/conda-content-trust_1693490622020/work
conda-libma

In [1]:
# bm25 테스트용
import pickle
from rank_bm25 import BM25Okapi
import json
from transformers import AutoTokenizer

class BM25Search:
    def __init__(self, embedding_path, wiki_path, bm25_tokenizer):
        self.tokenizer = AutoTokenizer.from_pretrained(bm25_tokenizer)

        # 위키 문서 로드
        try:
            with open(wiki_path, 'r', encoding='utf-8') as f:
                self.wiki_data = json.load(f)
            print("위키 파일 로드 성공")
        except Exception as e:
            print(f"위키 파일 로드 에러: {e}")
            raise e

        # 위키 문서의 텍스트를 추출 (context 부분만)
        try:
            self.contexts = [v['text'] for v in self.wiki_data.values()]
            print(f"Loaded {len(self.contexts)} documents from wiki data")
        except Exception as e:
            print(f"위키 데이터 프로세스 에러: {e}")
            raise e

        # BM25 모델 초기화 (embedding_path는 저장된 토크나이즈된 문서 리스트이다.)
        try:
            with open(embedding_path, 'rb') as file:
                tokenized_contexts = pickle.load(file)
            print("토크나이즈된 문서 리스트 ")

            # BM25 모델 초기화
            self.bm25 = BM25Okapi(tokenized_contexts)
            print("bm25 모델 초기화")
        except Exception as e:
            print(f"모델 초기화 오류: {e}")
            raise e

    def search_query(self, query, top_n=5):
        # 쿼리를 토큰화
        try:
            tokenized_query = self.tokenizer.tokenize(query)
            print(f"토크나이징된 쿼리: {tokenized_query}")
        except Exception as e:
            print(f"Error tokenizing query: {e}")
            raise e

        # BM25에서 점수를 계산
        try:
            doc_scores = self.bm25.get_scores(tokenized_query)
            top_indexes = sorted(range(len(doc_scores)), key=lambda i: doc_scores[i], reverse=True)[:top_n]
            return [(index, doc_scores[index]) for index in top_indexes]
        except Exception as e:
            print(f"bm25 점수 에러: {e}")
            raise e

    def get_document(self, index):
        try:
            return self.contexts[index]
        except IndexError as e:
            print(f"문서 index retreive 에러 {index}: {e}")
            return None

# BM25 임베딩 파일 및 위키 문서 경로
embedding_path = "/data/ephemeral/Jung/level2-mrc-nlp-10/Template/bm25_retrieval_result/BM25_embedding.bin"
wiki_path = "/data/ephemeral/data/wikipedia_documents.json"
bm25_tokenizer = "klue/bert-base"  # BERT 기반 토크나이저 사용

# BM25Search 클래스 인스턴스 생성
bm25_search = BM25Search(embedding_path, wiki_path, bm25_tokenizer)

# 쿼리 예시
query = '마오리언어와 영어, 뉴질랜드 수화를 공식 언어로 사용하는 나라는?'

# 쿼리에 대한 상위 5개의 문서 검색
try:
    results = bm25_search.search_query(query, top_n=5)
    for idx, score in results:
        document = bm25_search.get_document(idx)
        if document:
            print(f"Document index: {idx}, Score: {score:.4f}")
            print(f"Document content (first 300 characters): {document[:50]}...")
            print("=" * 50)
        else:
            print(f"문서를 찾지 못하였음 {idx}")
except Exception as e:
    print(f"에러 발생: {e}")



/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


위키 파일 로드 성공
Loaded 60613 documents from wiki data
토크나이즈된 문서 리스트 
bm25 모델 초기화
토크나이징된 쿼리: ['마오', '##리', '##언', '##어', '##와', '영어', ',', '뉴질랜드', '수화', '##를', '공식', '언어', '##로', '사용', '##하', '##는', '나라', '##는', '?']
Document index: 21753, Score: 67.6679
Document content (first 300 characters): 1826년 수크레는 볼리비아의 대통령으로 선출되었으나, 지역적인 정치 발전에 불만을 가지게...
Document index: 54901, Score: 58.2709
Document content (first 300 characters): 1872년 6월 13일 충북 옥천군 옥천읍 문정리에서 태어났다. 스무 살이 되던 해 부친이...
Document index: 54905, Score: 56.1938
Document content (first 300 characters): KBS의 밀정 방송 후 (사)범재 김규흥기념사업회는 보도에 반박하는 글을 홈페이지(http...
Document index: 4166, Score: 55.1116
Document content (first 300 characters): 합병 후, 1910년 11월 《동국역사》,《대한지지》등이 발매, 반포 금지 처분을 받았다....
Document index: 44244, Score: 54.4557
Document content (first 300 characters): 2010년 4월, 이란은 1기당 연간 10SWU의 능력을 가진 3세대 원심분리기 IR-4를...


In [1]:


import json
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

class TFIDFSearch:
    def __init__(self, embeddings_path, vectorizer_path, documents_path):
        # 임베딩 파일 로드
        with open(embeddings_path, 'rb') as f:
            self.tfidf_matrix = pickle.load(f)
        # TfidfVectorizer 로드
        with open(vectorizer_path, 'rb') as f:
            self.vectorizer = pickle.load(f)
        # 문서 데이터 로드
        with open(documents_path, 'r', encoding='utf-8') as f:
            self.documents = json.load(f)
            self.documents = [doc['text'] for doc in self.documents.values()]  # 문서 텍스트 리스트로 저장

    def search(self, query, top_n=5):
        # 쿼리를 TF-IDF 벡터로 변환
        query_vec = self.vectorizer.transform([query])
        # 문서와의 코사인 유사도 계산
        cosine_similarities = cosine_similarity(query_vec, self.tfidf_matrix)
        # 가장 유사한 문서의 인덱스를 찾기
        similar_indices = np.argsort(cosine_similarities.flatten())[-top_n:]
        # 가장 유사한 문서의 인덱스와 유사도 출력
        similar_documents = [(index, cosine_similarities.flatten()[index], self.documents[index]) for index in similar_indices]
        return similar_documents

# 인스턴스 생성 및 쿼리 실행
tfidf_search = TFIDFSearch(
    embeddings_path="/data/ephemeral/Jung/level2-mrc-nlp-10/Template/retrieval_result/Sparse_embedding.bin",
    vectorizer_path="/data/ephemeral/Jung/level2-mrc-nlp-10/Template/retrieval_result/TFIDF_vec.bin",
    documents_path="/data/ephemeral/data/wikipedia_documents.json"  # 문서 데이터 파일 경로 지정
)

# 쿼리 실행
query = '마오리언어와 영어, 뉴질랜드 수화를 공식 언어로 사용하는 나라는?'
results = tfidf_search.search(query)

# 결과 출력
for index, score, text in sorted(results, key=lambda x: x[1], reverse=True):
    print(f"문서 index: {index}, Score: {score:.4f}")
    print(f"Context: {text[:300]}...")  # 문서의 일부만 출력
    print("=" * 50)



/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


문서 index: 10676, Score: 0.2492
Context: 다음과 같이 트랜스폰더(transponder) 모드가 존재한다.

모드 1: 2디지트, 5비트 임무 코드를 제공한다. (군전용, 조종석에서 선택)

모드 2: 4디지트, 8진법으로 제공한다. (군전용, 특정 항공기 타입에 따라서 지상 또는 조종석에서 선택)

모드 3/A: 4디지트, 8진법 식별코드를 ATC(군 및 민간)에서 정한 항공기(일명 Squawk Code)에게 제공한다.

모드 4: 3펄스로 암호화 코드를 제공한다. (군전용)

모드 5: Mode S와 유사한 암호화 능력을 제공하며, ADS-B와 GPS 위치를 전송을 포함...
문서 index: 21753, Score: 0.2204
Context: 1826년 수크레는 볼리비아의 대통령으로 선출되었으나, 지역적인 정치 발전에 불만을 가지게 되었다. 1828년 볼리바르와 그의 지지자, 그리고 볼리비아를 위해 그가 쓴 헌법에 반대하는 운동이 거세게 일어나자 수크레는 사임하고 아내의 고향 도시인 키토로 떠났다. 그는 정치를 불편하게 여겼으며, 은퇴할 생각을 가졌다.

1828년 말 볼리바르의 요청으로 그란콜롬비아 의회는 그를 의회 의장으로 지명하였다. 이들은 향후 볼리바르의 후계자로 수크레를 공화국 대통령으로 지명하고자 하였으나, 수크레가 거부할 것 같아서 실행하지 않았다.

수크레...
문서 index: 44238, Score: 0.1936
Context: 구엔돌로에나(Guendoloēna, 구엔돌렌)는 몬머스의 제프리의 《브리타니아 열왕사》에 나오는 브리튼인의 전설적인 여왕이다. 물론 《브리타니아 열왕사》에 올라온 다른 군주들이 그러하듯이 구엔돌로에나는 야사의 존재일 뿐 실존인물이라는 증거는 없다.

초대 콘월 공작 코리네우스의 딸로 태어났다. 초대 브리튼 왕 브루투스의 장남 로크리누스와 결혼해 그 사이에 아들 마단을 낳았다. 로크리누스는 게르만족 공주 에스트릴디스를 사랑했고 외도 끝에 이미 그 사이에 딸 하브렌을 낳았다. 때문에 그

In [4]:
#bm25 test
import importlib
import retrieval
importlib.reload(retrieval)

# BM25Search 인스턴스 생성
bm25 = retrieval.BM25Search()

# BM25 임베딩 생성 및 저장/로드
bm25.get_sparse_embedding()

# 쿼리 검색 수행
result = bm25.search_query_bm25()


Data route: /data/ephemeral/data/train_dataset/
Test data route: /data/ephemeral/data/test_dataset/
Wiki route: /data/ephemeral/data/wikipedia_documents.json
Data path: ./bm25_retrieval_result


loading configuration file config.json from cache at /data/ephemeral/home/.cache/huggingface/hub/models--klue--bert-base/snapshots/77c8b3d707df785034b4e50f2da5d37be5f0f546/config.json
Model config BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading file vocab.txt from cache at /data/ephemeral/home/.cache/huggingface/hub/models--klue--bert-base/snapshots/77c8b3d707df785034b4e50f2da5d37be5f0f546/vocab.txt
loading file tokenizer.json

File loaded successfully
위키 컨텍스트의 수: 56737
BM25 임베딩을 로드했습니다.


BM25 retrieval: 100%|██████████| 600/600 [00:00<00:00, 12499.48it/s]


In [4]:
import importlib
import Dense  
import torch
importlib.reload(Dense)

# DenseRetrievalSearch 인스턴스 생성
dense_retrieval = Dense.DenseRetrievalSearch(
    model_class=Dense.DenseRetrievalModel,
    model_path='./retrieval_result/dense_retrieval_model.pth',  # 모델 가중치 파일 경로
    tokenizer_name='klue/bert-base',  # 사용한 토크나이저와 동일해야 함
    embeddings_path='./retrieval_result/Dense_embedding.npy',  # 문서 임베딩 파일 경로
    doc_ids_path='./retrieval_result/doc_ids.json',  # 문서 ID 파일 경로
    contexts_path='./retrieval_result/contexts.json',  # 문서 내용 파일 경로
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

# Dense 임베딩 생성 및 저장/로드
dense_retrieval.get_dense_embedding()  # 현재는 임베딩이 이미 로드되어 있다고 가정

# 쿼리 검색 수행
query = '마오리언어와 영어, 뉴질랜드 수화를 공식 언어로 사용하는 나라는?'
results = dense_retrieval.search_query_dense(query, top_n=10)

# 검색 결과 출력
for idx, score in results:
    document = dense_retrieval.get_document(idx)
    if document:
        print(f"Document index: {idx}, Score: {score:.4f}")
        print(f"Document content (first 50 characters): {document[:50]}...")
        print("=" * 50)
    else:
        print(f"문서를 찾지 못하였음 {idx}")


Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions

모델과 토크나이저가 로드되었습니다.
문서 임베딩이 ./retrieval_result/Dense_embedding.npy에서 로드되었습니다. 임베딩 shape: (60613, 768)
문서 ID와 내용이 로드되었습니다. 총 문서 수: 60613
문서 임베딩이 정규화되었습니다.
Dense 임베딩을 로드했습니다.
쿼리 임베딩이 생성되고 정규화되었습니다.
쿼리에 대한 상위 10개 문서를 Dense Retrieval으로 검색했습니다.
Document index: 39838, Score: 0.6234
Document content (first 50 characters): 영어, 말레이어, 중국어, 카다잔두순어 등이 사용되었으나, 실질적인 공용어는 영어였다. 영...
Document index: 185, Score: 0.6190
Document content (first 50 characters): 현재 공용어는 스페인어(쿠바 방식의 스페인어)이다. 하지만 관광 산업에 주력하고 있는 미국...
Document index: 4739, Score: 0.6190
Document content (first 50 characters): 현재 공용어는 스페인어(쿠바 방식의 스페인어)이다. 하지만 관광 산업에 주력하고 있는 미국...
Document index: 58774, Score: 0.6091
Document content (first 50 characters): 농아인 공동체는 멕시코 수화와 유카탄 수화를 사용하고, 바하칼리포르니아주 북부에서는 미국 ...
Document index: 2652, Score: 0.6051
Document content (first 50 characters): 이 언어는 말레이시아의 실질적인 공용어이며, 말레이어의 일종이다. 이름을 말레이시아어라고 ...
Document index: 3622, Score: 0.6005
Document content (first 50 characters): 아일랜드어(Gaeilge)는 인도유럽어족 켈트어파에 속하는 

In [2]:
!pip install transformers==4.24.0

  Obtaining dependency information for transformers==4.24.0 from https://files.pythonhosted.org/packages/a4/df/3248eac2923ceffdf55686ff318e002b558e7c51f6a909dd870cf3185949/transformers-4.24.0-py3-none-any.whl.metadata
  Using cached transformers-4.24.0-py3-none-any.whl.metadata (90 kB)
  Obtaining dependency information for tokenizers!=0.11.3,<0.14,>=0.11.1 from https://files.pythonhosted.org/packages/94/60/ff26cce378023624ffcad91edaa4871f561d6ba7295185c45037ddba80e2/tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
Using cached transformers-4.24.0-py3-none-any.whl (5.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 6.3 MB/s eta 0:00:0000:0100:010m
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.0
    Uninstalling tokenizers-0.20.0:
      Successfully uninstalled tokenizers-0.20.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.2
    Uninstalling transformers-4.45

In [5]:
# Dense Retrieval을 위한 Dense.py 모듈 임포트
import importlib
import Dense  # Dense.py 파일이 동일한 디렉토리에 있다고 가정
importlib.reload(Dense)

# DenseRetrievalSearch 인스턴스 생성
dense_retrieval = Dense.DenseRetrievalSearch(
    model_class=Dense.DenseRetrievalModel,
    model_path='/data/ephemeral/retrieval_result/dense_retrieval_model.pth',  # 모델 가중치 파일 경로
    tokenizer_name='klue/bert-base',  # 사용한 토크나이저와 동일해야 함
    embeddings_path='/data/ephemeral/retrieval_result/Dense_embedding.npy',  # 문서 임베딩 파일 경로
    doc_ids_path='/data/ephemeral/retrieval_result/doc_ids.json',  # 문서 ID 파일 경로
    contexts_path='/data/ephemeral/retrieval_result/contexts.json',  # 문서 내용 파일 경로
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

# Dense 임베딩 생성 및 저장/로드
dense_retrieval.get_dense_embedding()  # 현재는 임베딩이 이미 로드되어 있다고 가정

# 테스트 데이터셋 경로 설정
test_data_route = '/data/ephemeral/data/test_dataset'  # 실제 테스트 데이터셋 경로로 수정

# Dense Retrieval을 통해 검색하고 DatasetDict 생성
result = dense_retrieval.search_and_create_dataset_dense(test_data_route=test_data_route, k=20)

print(result)


Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions

모델과 토크나이저가 로드되었습니다.
문서 임베딩이 /data/ephemeral/retrieval_result/Dense_embedding.npy에서 로드되었습니다. 임베딩 shape: (60613, 768)
문서 ID와 내용이 로드되었습니다. 총 문서 수: 60613
문서 임베딩이 정규화되었습니다.
Dense 임베딩을 로드했습니다.
테스트 데이터셋의 총 쿼리 수: 600


Dense Retrieval:   0%|          | 0/600 [00:00<?, ?it/s]

쿼리 임베딩이 생성되고 정규화되었습니다.
쿼리에 대한 상위 20개 문서를 Dense Retrieval으로 검색했습니다.
쿼리 임베딩이 생성되고 정규화되었습니다.
쿼리에 대한 상위 20개 문서를 Dense Retrieval으로 검색했습니다.
쿼리 임베딩이 생성되고 정규화되었습니다.


Dense Retrieval:   1%|          | 5/600 [00:00<00:13, 42.52it/s]

...

쿼리에 대한 상위 20개 문서를 Dense Retrieval으로 검색했습니다.
쿼리 임베딩이 생성되고 정규화되었습니다.
쿼리에 대한 상위 20개 문서를 Dense Retrieval으로 검색했습니다.
쿼리 임베딩이 생성되고 정규화되었습니다.
쿼리에 대한 상위 20개 문서를 Dense Retrieval으로 검색했습니다.
쿼리 임베딩이 생성되고 정규화되었습니다.
쿼리에 대한 상위 20개 문서를 Dense Retrieval으로 검색했습니다.
쿼리 임베딩이 생성되고 정규화되었습니다.
쿼리에 대한 상위 20개 문서를 Dense Retrieval으로 검색했습니다.
쿼리 임베딩이 생성되고 정규화되었습니다.
쿼리에 대한 상위 20개 문서를 Dense Retrieval으로 검색했습니다.
쿼리 임베딩이 생성되고 정규화되었습니다.
쿼리에 대한 상위 20개 문서를 Dense Retrieval으로 검색했습니다.


Dense Retrieval:  99%|█████████▉| 596/600 [00:14<00:00, 36.98it/s]

쿼리 임베딩이 생성되고 정규화되었습니다.
쿼리에 대한 상위 20개 문서를 Dense Retrieval으로 검색했습니다.


Dense Retrieval: 100%|██████████| 600/600 [00:14<00:00, 40.38it/s]


쿼리 임베딩이 생성되고 정규화되었습니다.
쿼리에 대한 상위 20개 문서를 Dense Retrieval으로 검색했습니다.
쿼리 임베딩이 생성되고 정규화되었습니다.
쿼리에 대한 상위 20개 문서를 Dense Retrieval으로 검색했습니다.
쿼리 임베딩이 생성되고 정규화되었습니다.
쿼리에 대한 상위 20개 문서를 Dense Retrieval으로 검색했습니다.
쿼리 임베딩이 생성되고 정규화되었습니다.
쿼리에 대한 상위 20개 문서를 Dense Retrieval으로 검색했습니다.
DataFrame columns: ['question', 'id', 'context']
DatasetDict가 성공적으로 생성되었습니다.
DatasetDict({
    validation: Dataset({
        features: ['context', 'id', 'question'],
        num_rows: 600
    })
})


In [4]:
from MRC import Extraction_based_MRC
model = Extraction_based_MRC()
model.train() #model.train()은 기존에 주어진 데이터셋을 활용하여 훈련합니다. model.train(train_dataset, eval_dataset)으로 본인이 직접 지정한 데이터를 훈련할 수도 있습니다.
# 이때, train 데이터셋의 형태는 load_from_disk(기존에 주어진 데이터경로)['train'], eval 데이터셋의 형태는 load_from_disk(기존에 주어진 데이터경로)['validation']의 형태여야 합니다.
# 모델은 Extraction_based_MRC_outputs + model_name의 폴더로 저장됩니다.
# 모델은 학습이 끝나면 가장 낮은 validaiton loss를 기준으로 저장됩니다.
model.inference(result)
# retrieval을 통해 얻어낸 결과를 predict_reuslt에 json파일 형태로 저장합니다.
"""inference는 반드시
DatasetDict({
    validation: Dataset({
        features: ['context', 'id', 'question'],
        num_rows: 600
    })
}) 의 형태를 따라야 합니다.
"""


loading configuration file config.json from cache at /data/ephemeral/home/.cache/huggingface/hub/models--klue--bert-base/snapshots/77c8b3d707df785034b4e50f2da5d37be5f0f546/config.json
Model config BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading configuration file config.json from cache at /data/ephemeral/home/.cache/huggingface/hub/models--klue--bert-base/snapshots/77c8b3d707df785034b4e50f2da5d37be5f0f546/config.json
Model con

PyTorch: setting up devices
Map: 100%|██████████| 3952/3952 [00:13<00:00, 299.06 examples/s]


train_dataset을 넣지 않아 기존에 주어진 train dataset으로 학습합니다.


Map: 100%|██████████| 240/240 [00:01<00:00, 238.51 examples/s]
***** Running training *****
  Num examples = 7978
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1996
  Number of trainable parameters = 110028290
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


eval_dataset을 넣지 않아 기존에 주어진 eval_dataset으로 평가합니다.


...

Saving model checkpoint to /data/ephemeral/home/practice/Template/Extraction_based_MRC_outputsbert-base/checkpoint-1996
Configuration saved in /data/ephemeral/home/practice/Template/Extraction_based_MRC_outputsbert-base/checkpoint-1996/config.json
Model weights saved in /data/ephemeral/home/practice/Template/Extraction_based_MRC_outputsbert-base/checkpoint-1996/pytorch_model.bin
tokenizer config file saved in /data/ephemeral/home/practice/Template/Extraction_based_MRC_outputsbert-base/checkpoint-1996/tokenizer_config.json
Special tokens file saved in /data/ephemeral/home/practice/Template/Extraction_based_MRC_outputsbert-base/checkpoint-1996/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /data/ephemeral/home/practice/Template/Extraction_based_MRC_outputsbert-base/checkpoint-998 (score: 0.9768912196159363).
Map: 100%|██████████| 600/600 [00:03<00:00, 166.23 examples/s]
The following columns in the tes

post_process_function을 실행합니다.


100%|██████████| 600/600 [00:18<00:00, 31.58it/s]


Dataset({
    features: ['context', 'id', 'question', 'answers'],
    num_rows: 600
})
output을 id : answers 형태의 json파일로 내보냅니다. 결과는 model.extraction_mrc_results로 확인할 수 있습니다.


"inference는 반드시\nDatasetDict({\n    validation: Dataset({\n        features: ['context', 'id', 'question'],\n        num_rows: 600\n    })\n}) 의 형태를 따라야 합니다.\n"

In [6]:
from MRC import Extraction_based_MRC
model = Extraction_based_MRC()
model.load_model() # 체크포인트의 제일 마지막 폴더를 따라가서 trainer_state.json을 열고 거기서 best_model_checkpoint의 경로를 통해 모델을 불러옵니다.
model.inference(result)
# 불러온 모델은 inference할 수 있습니다.

[nltk_data] Downloading package punkt to
[nltk_data]     /data/ephemeral/home/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some 

제일 마지막 checkpoint : /data/ephemeral/home/practice/Template/Extraction_based_MRC_outputsbert-base/checkpoint-998/trainer_state.json
best checkpoint : /data/ephemeral/home/practice/Template/Extraction_based_MRC_outputsbert-base/checkpoint-998
bestmodel 체크포인트로 모델과 Trainer가 로드되었습니다.


Map: 100%|██████████| 600/600 [00:16<00:00, 36.71 examples/s]
The following columns in the test set don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id. If offset_mapping, example_id are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 19936
  Batch size = 8


post_process_function을 실행합니다.


100%|██████████| 600/600 [01:34<00:00,  6.34it/s]


Dataset({
    features: ['context', 'id', 'question'],
    num_rows: 600
})
output을 id : answers 형태의 json파일로 내보냅니다. 결과는 model.extraction_mrc_results로 확인할 수 있습니다.


In [ ]:
import All_dataset
from arguments import Extraction_based_MRC_arguments, Dense_search_retrieval_arguments, Generation_based_MRC_arguments
datas = All_dataset.prepare_dataset(Extraction_based_MRC_arguments)
train = datas.get_mrc_train_dataset()
eval = datas.get_mrc_eval_dataset()
# test =  datas.get_mrc_test_dataset(리트리버결과)
print(train)
print(eval)